## 9. Базы данных (с уклоном в PostgreSQL)

![Database](https://raw.githubusercontent.com/amaargiru/pycore/main/pics/09_Database.png)  

### БД и СУБД

База данных (БД) — набор взаимосвязанных данных, хранение, доступ и обработка которых осуществляется посредством [систем управления базами данных (СУБД)](https://en.wikipedia.org/wiki/Database#Database_management_system).

СУБД отвечает за:  
• поддержку языка запросов БД;  
• хранение данных;  
• организацию и оптимизацию процессов записи и извлечения данных.

Отталкиваясь от способа доступа к БД, СУБД можно разделить на три класса:  
• файл-серверные. Файлы хранятся централизованно на сервере, СУБД установлены на клиентских машинах. Примеры — Microsoft Access, Visual FoxPro;  
• клиент-серверные. И файлы данных, и СУБД располагаются централизованно на сервере. Примеры — MySQL, PostgreSQL, Oracle Database, MS SQL Server, MariaDB(форк MySQL);  
• встраиваемые. Встраиваемая СУБД предназначена, как правило, для работы с данными одного конкретного приложения и не рассчитана на работу с внешними запросами. Реализуется, как правило, не в виде самостоятельно инсталлируемого ПО, а в виде подключаемой библиотеки. Самый известный практический пример — [SQLite](https://en.wikipedia.org/wiki/SQLite), остальные встраиваемые СУБД гораздо менее распространены.

Обобщенные требования к РСУБД изложены математиком Эдгаром Коддом в [«12 правилах Кодда»](https://en.wikipedia.org/wiki/Codd%27s_12_rules).

Далее мы сосредоточимся на СУБД PostgreSQL. Также, если не указано иное, при рассмотрении примеров SQL-запросов будет использоваться синтаксис PostgreSQL.


### Транзакция

Остановимся чуть более подробно на механизме транзакций, на требованиях, предъявляемых к транзакциям и на методах соблюдения этих требований.

Формально, [транзакция](https://en.wikipedia.org/wiki/Database_transaction) — неделимая последовательность действий (атомарная операция, группа операций, которые выполняются как единое целое), обеспечивает выполнение либо всех действий из последовательности, либо ни одного. Если в ходе выполнения транзакции произошел сбой, состояние системы должно быть возвращено к исходному, уже выполненные действия должны быть отменены.

Если же говорить неформально, поток транзакций — это динамическое изменение состояний базы данных, схожее с сердцебиением или дыханием у человека. Также как для человека, правильное, согласованное сердцебиение является залогом нормального функционирования; разлад потока транзакций может привести с существенным сбоям и падению базы данных.

Канонический пример — списывание денег с одного счета и зачисление на другой, для чего необходимы два процесса проведения изменений, которые гарантированно должны выполниться или не выполниться вместе. Зачисление денег на один счет без списывания со второго, или наоборот, списывание денег с одного счета без изменений на втором — совершенно недопустимая операция, ставящая под сомнение самое существование жизни, вселенной и всего такого.

В частном случае в группе транзакций может быть одна операция.

В Postgre транзакция начинается с команды BEGIN и заканчивается командой COMMIT либо отменяется командой ROLLBACK.

Транзакция — способ группировки приложением нескольких операций записи и чтения в одну логическую единицу. По сути, все операции записи и чтения в ней выполняются как одна: вся транзакция или целиком выполняется успешно (с фиксацией изменений), или целиком завершается неудачно (с прерыванием и откатом). Во втором случае приложение может спокойно попробовать выполнить её еще раз. Транзакции значительно упрощают для приложения обработку ошибок, поскольку нет нужды заботиться о частичных отказах, то есть случаях, когда часть операций завершилась успешно, а часть — нет (неважно, по каким причинам).

По сути, транзакции - универсальный механизм реагирования на разнородные проблемы:  
программное или аппаратное обеспечение базы данных может отказать в любой момент (в том числе посередине операции записи);  
в любой момент может произойти фатальный сбой приложения (в том числекогда последовательность операций выполнена наполовину);  
разрывы сети могут неожиданно отрезать приложение от базы данных или один узел базы от другого;  
несколько клиентов могут производить операции записи одновременно, перезаписывая изменения друг друга;  
клиент может прочитать данные, которые не имеют смысла, поскольку были обновлены только частично;  
состояния гонки между клиентами могут привести к неожиданным ошибкам.  

Транзакции - не закон природы, они были созданы с определенной целью, а именно для упрощения модели программирования приложений, работающих с базами данных. Благодаря использованию транзакций приложение получает возможность игнорировать определенные сценарии ошибок и вопросы конкурентного доступа, поскольку вместо него этим занимается база (данный аспект называется гарантиями функциональной безопасности).

В конце 2000-х годов приобрели популярность нереляционные (NoSQL) базы данных. Их целью было улучшить существующее положение дел с реляционными БД с помощью новых моделей данных, репликации и секционирования. Транзакции оказались главной жертвой этого новшества: многие базы нового поколения полностью от них отказались или поменяли значение термина: теперь он стал у них означать намного более слабый набор функциональных гарантий, чем ранее.

Вместе с шумихой вокруг этого нового обильного урожая распределенных баз данных стало широко распространяться мнение, что транзакции — антоним масштабируемости и любой крупномасштабной системе необходимо отказаться от них ради сохранения хорошей производительности и высокой доступности. С другой стороны, производители БД иногда представляют транзакционные функциональные гарантии как обязательное требование для «серьезных приложений», оперирующих «ценными данными». Обе точки зрения — чистейшей воды преувеличение.

Истина не столь проста: как и любое другое техническое проектное решение, транзакции имеют свои достоинства и ограничения. Чтобы лучше разобраться в их плюсах и минусах, глубже заглянем в подробности предоставляемых транзакциями функциональных гарантий — как при обычной эксплуатации, так и в различных нестандартных (хотя и реалистичных) обстоятельствах.

### ACID

Обеспечиваемые транзакциями гарантии функциональной безопасности часто описываются известной аббревиатурой ACID (atomicity, consistency, isolation, durability).

Аббревиатура был придумана в 1983 году Тео Хэрдером и Андреасом Ройтером в попытке создать четкую терминологию для механизмов обеспечения отказоустойчивости в базах данных. Однако на практике реализации ACID в разных базах отличаются друг от друга. Например, существуют серьезные различия в понимании термина «изоляция», так что на сегодняшний день заявление о «совместимости системы с ACID» не дает четкого представления о предоставляемых гарантиях. К сожалению, ACID стал скорее термином из области маркетинга.

1. **Атомарность** (atomicity). Это свойство означает, что либо транзакция будет зафиксирована в базе данных полностью, т. е. будут зафиксированы результаты выполнения всех ее операций, либо не будет зафиксирована ни одна операция транзакции.

2. **Согласованность** (consistency). Это свойство предписывает, чтобы в результате успешного выполнения транзакции база данных была переведена из одного консистентного состояния в другое консистентное состояние.

Идея согласованности в смысле ACID состоит в том, что определенные утверждения относительно данных (инварианты) должны всегда оставаться справедливыми; например, в системе бухгалтерского учета кредит всегда должен сходиться с дебетом по всем счетам. Если транзакция начинается при допустимом (в соответствии с этими инвариантами) состоянии базы данных и любые производимые во время транзакции операции записи сохраняют это свойство, то можно быть уверенными, что система всегда будет удовлетворять инвариантам.

3. **Изолированность** (isolation). Во время выполнения транзакции другие транзакции должны оказывать по возможности минимальное влияние на нее.

Изоляция в смысле ACID означает, что конкурентно выполняемые транзакции изолированы друг от друга — они не могут помешать друг другу, и каждая транзакция выполняется так, будто она единственная во всей базе. БД гарантирует, что результат фиксации транзакций такой же, как если бы они выполнялись последовательно, хотя в реальности они могут выполняться конкурентно.

4. **Долговечность**, **Сохраняемость** или **Устойчивость** (durability, перевод разнится даже в авторитетных источниках, например, в "Википедии" и в русском переводе книги Мартина Клеппмана "Высоконагруженные приложения"). После успешной фиксации транзакции пользователь должен быть уверен, что данные надежно сохранены в базе данных и впоследствии могут быть извлечены из нее, независимо от последующих возможных сбоев в работе системы.

Именно через призму соблюдения требований ACID (или отказа от их части) нужно рассматривать вопросы надежности, предсказуемости и масштабируемости БД.

На практике реализации ACID в разных базах отличаются друг от друга. Например, существуют серьезные различия в понимании термина «изоляция». Идея в целом правильная, но дьявол, как обычно, кроется в деталях. На сегодняшний день заявление о «совместимости системы с ACID» не дает четкого представления о предоставляемых гарантиях. К сожалению, ACID
стал скорее термином из области маркетинга.

Атомарность, изоляция и сохраняемость — свойства базы данных, в то время как согласованность (в смысле ACID) — свойство приложения. Оно может полагаться на свойства атомарности и изоляции базы данных, чтобы обеспечить согласованность, но не на одну только базу. Следовательно, букве C в каком-то смысле на самом деле не место в аббревиатуре ACID. Буква C была добавлена в ACID в статье Хэрдера и Ройтера "Principles of Transaction-Oriented Database Recovery" для красоты аббревиатуры и не считалась в то время (1983 год) чем-то важным.


### Проблемы параллельного доступа с использованием транзакций

Транзакции можно исполнять последовательно или параллельно. И если в первом случае все более-менее понятно и предсказуемо, то в случае параллельного исполнения транзакций возможны следующие проблемы:

**Фантомное чтение** (phantom reads) — одна транзакция в ходе своего выполнения несколько раз выбирает множество строк по одним и тем же критериям. Другая транзакция в интервалах между этими выборками добавляет строки или изменяет столбцы некоторых строк, используемых в критериях выборки первой транзакции, и успешно заканчивается. В результате получится, что одни и те же выборки в первой транзакции дают разные множества строк.

**Неповторяющееся чтение** (non-repeatable read) — при повторном чтении в рамках одной транзакции ранее прочитанные данные оказываются изменёнными.

**«Грязное» чтение** (dirty read) — чтение данных, добавленных или изменённых транзакцией, которая впоследствии не подтвердится (откатится);

**Потерянное обновление** (lost update) — при одновременном изменении одного блока данных разными транзакциями теряются все изменения, кроме последнего.

### Уровни изоляции транзакций

Для борьбы с проблемами, порождаемыми параллельным исполнением транзакций у нас есть соответствующий инструмент — [уровни изоляции транзакций](https://en.wikipedia.org/wiki/Isolation_(database_systems)) — фактически, выбор между скоростью работы и обеспечением согласованности данных, т. к. при выполнении параллельных транзакций в СУБД всегда допускается получение несогласованных данных, и разработчик должен найти баланс между количеством параллельных транзакций и согласованностью данных.

Стандарт SQL-92 определяет шкалу из четырёх уровней изоляции: чтение незафиксированных данных, чтение фиксированных данных, повторяющееся чтение, упорядочивание. Первый из них является самым слабым, последний — самым сильным, каждый последующий включает в себя все предыдущие.

**Чтение незафиксированных данных (read uncommitted)**

Низший (первый) уровень изоляции. Если несколько параллельных транзакций пытаются изменять одну и ту же строку таблицы, то в окончательном варианте строка будет иметь значение, определенное всем набором успешно выполненных транзакций. При этом возможно считывание не только логически несогласованных данных, но и данных, изменения которых ещё не зафиксированы, т. к. транзакции, выполняющие только чтение, при данном уровне изоляции никогда не блокируются. Данные блокируются на время выполнения команды записи, что гарантирует, что команды изменения одних и тех же строк, запущенные параллельно, фактически выполнятся последовательно, и ни одно из изменений не потеряется.

**Чтение фиксированных данных (read committed)**

Большинство СУБД, в частности, Microsoft SQL Server, PostgreSQL и Oracle, по умолчанию используют именно этот уровень. На этом уровне обеспечивается защита от чтения промежуточных данных, тем не менее, в процессе работы одной транзакции другая может быть успешно завершена и сделанные ею изменения зафиксированы. В итоге первая транзакция будет работать с другим набором данных.  

Метод read committed реализуется либо при помощи блокировки данных на чтение во время записи (теряем время), либо на хранении копии данных, снятой до начала записи (теряем ОЗУ).

**Повторяющееся чтение (repeatable read)**

Уровень, при котором читающая транзакция блокирует изменения данных, прочитанных ею ранее. При этом никакая другая транзакция не может изменять данные, читаемые текущей транзакцией, пока та не окончена.

**Упорядочивание (serializable)**

Самый высокий уровень изолированности; транзакции полностью изолируются друг от друга, каждая выполняется так, как будто параллельных транзакций не существует. Только на этом уровне параллельные транзакции не подвержены эффекту «фантомного чтения».

<style>
table th:first-of-type {
    width: 20%;
}
table th:nth-of-type(2) {
    width: 20%;
}
table th:nth-of-type(3) {
    width: 20%;
}
table th:nth-of-type(4) {
    width: 20%;
}
table th:nth-of-type(5) {
    width: 20%;
}
</style>

| Уровень изоляции | Фантомное чтение| Неповторяющееся чтение | «Грязное» чтение | Потерянное обновление |
| :- | :-: | :-: | :-: | :-: |
| Отсутствие изоляции | + | + | + | + |
| Read uncommitted | + | + | + | - |
| Read committed | + | + | - | - |
| Repeatable read | + | - | - | - |
| Serializable | - | - | - | - |

Задача любой кластерной СУБД — разделение данных между узлами таким образом, чтобы для пользователя они представляли единое целое, при этом система могла использовать мощности всех узлов одновременно.

### Репликация БД

Репликация баз данных может использоваться во многих СУБД, обычно в режиме “ведущий–ведомый”, где роль ведущего сервера играет оригинал (master), а его копии являются ведомыми (slave).

Ведущая база данных обычно поддерживает только операции записи. Ведомые БД получают от ведущей копии ее содержимого и поддерживают только операции чтения. Все команды для модификации данных, такие как вставка, удаление или обновление, должны направляться ведущей базе данных. В большинстве приложений чтение происходит намного чаще, чем запись, поэтому ведомых БД обычно больше, чем ведущих.

Преимущества репликации базы данных:  
Повышенная производительность. В модели «ведущий–ведомый» все операции записи и обновления происходят на ведущих узлах, а операции чтения распределяются между ведомыми. Это улучшает производительность, увеличивая количество запросов, которые можно обрабатывать параллельно.  
Надежность. Если один из ваших серверов с базой данных сломается из-за стихийного бедствия, такого как тайфун или землетрясение, данные не будут утеряны. Вам не нужно беспокоиться о потере данных, так как они реплицируются по разным местам.  
Высокая доступность. За счет репликации данных по разным местам ваш веб-сайт будет продолжать работать, даже если одна из БД выйдет из строя, поскольку у вас по-прежнему будет доступ к данным, размещенным на другом сервере.  

Три популярных алгоритма репликации изменений между узлами: репликация с одним ведущим узлом (single-leader), с несколькими ведущими узлами (multi-leader) и без ведущего узла (leaderless).

### Репликация с одним ведущим узлом

Схема ее работы следующая.

1. Одна из реплик назначается ведущим (master) узлом. Клиенты, желающие записать данные в базу, должны отправить свои запросы ведущему узлу, который сначала записывает новые данные в свое локальное хранилище.

2. Другие реплики называются ведомыми (followers) узлами. Всякий раз, когда ведущий узел записывает в свое хранилище новые данные, он также отправляет информацию об изменениях данных всем ведомым узлам в качестве части журнала репликации (replication log) или потока изменений (change stream). Все ведомые узлы получают журнал от ведущего и обновляют соответствующим образом свою локальную копию БД, применяя все операции записи в порядке их обработки ведущим узлом.

3. Когда клиенту требуется прочитать данные из базы, он может выполнить запрос или к ведущему узлу, или к любому из ведомых. Однако запросы на запись разрешено отправлять только ведущему (ведомые с точки зрения клиента предназначены только для чтения).

Такой режим репликации — встроенная возможность многих реляционных баз данных, например PostgreSQL (начиная с версии 9.0) или MySQL. MongoDB и Kafka тоже имеют такие возможности.

Создание новых ведомых узлов.

Время от времени приходится создавать новые ведомые узлы — с целью увеличить количество реплик или заменить сбойные узлы. Как же гарантировать, что новый ведомый узел будет содержать точную копию данных ведущего?

Процесс выглядит следующим образом.

1. Сделать согласованный снимок состояния БД ведущего узла на определенный момент времени — по возможности без блокировки всей базы. В большинстве баз такая возможность есть, так как она нужна и для резервного копирования. В некоторых случаях понадобятся сторонние утилиты, например innobackupex для СУБД MySQL.

2. Скопировать снимок состояния на новый ведомый узел.

3. Ведомый узел подключается к ведущему и запрашивает все изменения данных, произошедшие с момента создания снимка. Для этого нужно, чтобы снимок состояния соотносился с определенной позицией в журнале репликации ведущего узла. Сама позиция называется по-разному: в PostgreSQL — регистрационным номером транзакции в журнале (log sequence number), в MySQL — координатами в бинарном журнале (binlog coordinates).

4. Когда ведомый узел завершил обработку изменений данных, произошедших с момента снимка состояния, говорят, что он наверстал упущенное. После этого он может
продолжать обрабатывать поступающие от ведущего узла изменения данных.

На практике создание и настройка ведомого узла очень зависит от используемой базы данных. В ряде систем этот процесс полностью автоматизирован, а в других — представляет собой запутанную многошаговую последовательность действий, которую должен вручную выполнить администратор.

Отказ ведомого узла: наверстывающее восстановление

Каждый ведомый узел хранит на своем жестком диске журнал полученных от ведущего изменений данных. В случае сбоя и перезагрузки ведомого узла или временного прекращения работы участка сети между ведущим и ведомым узлами последний может легко возобновить работу: из своего журнала он знает, какая транзакция была обработана перед сбоем. Следовательно, ведомый узел способен подключиться к ведущему и запросить все изменения данных, имевшие место за то время, пока он был недоступен.

Отказ ведущего узла: восстановление после отказа

Справиться с отказом ведущего узла сложнее: необходимо «повысить в звании» один из ведомых до ведущего, настроить клиенты на отправку записей новому ведущему, а другие ведомые должны начать получать изменения данных от нового ведущего. Этот процесс называется восстановлением после отказа (failover).

Восстановление после отказа может выполняться вручную (администратор получает оповещение об отказе ведущего узла и принимает соответствующие меры по созданию нового ведущего) или автоматически. Автоматически процесс восстановления после отказа обычно состоит из следующих шагов.

1. Установить отказ ведущего узла. Многое может потенциально пойти не так: фатальные сбои, перебои питания, сетевые проблемы и т. д. Не существует надежного способа определить, что именно пошло не так. Поэтому во многих системах для данной цели используется превышение времени ожидания: узлы постоянно обмениваются сообщениями друг с другом, и если один из них не отвечает в течение определенного времени (скажем, 30 секунд), то считается, что он не работает (это не относится к случаю преднамеренного отключения ведущего узла для запланированного техобслуживания).

2. Выбрать новый ведущий узел. Здесь можно задействовать процесс «выборов» (ведущий узел выбирается в соответствии с большинством оставшихся реплик), или же этот ведущий назначает предварительно выбранный узел-контроллер (controller node). Оптимальным кандидатом на роль нового ведущего узла обычно является реплика с наиболее свежими изменениями данных, полученными от старого (в целях минимизации потерь данных). Согласование нового ведущего между всеми узлами — задача консенсуса, которую мы обсудим позже.

3. Настроить систему на использование нового ведущего узла. Клиенты должны начать отправлять запросы на запись новому ведущему узлу (мы обсудим это позже). Если старый ведущий узел возобновляет работу, может оказаться, что он продолжает считать себя ведущим и не осознает решение остальных реплик считать его недееспособным. Система должна обеспечить превращение старого ведущего узла в ведомый и признание им нового ведущего.

### Репликация с несколькими ведущими узлами

Сравним схему с одним и несколькими ведущими узлами в смысле развертывания в нескольких ЦОДах.

Производительность. При схеме с одним ведущим узлом каждая операция записи должна проходить через Интернет в ЦОД, содержащий ведущий узел. Это может существенно задержать операцию записи и вообще пойдет вразрез с идеей нескольких ЦОДов. При схеме с несколькими ведущими узлами все операции записи будут обрабатываться в локальных ЦОДах и реплицироваться асинхронно в остальные ЦОДы. Таким образом, сетевые задержки между ЦОДами становятся незаметными для пользователей, а значит, субъективная производительность возрастет.

Устойчивость к перебоям в обслуживании ЦОДов. При схеме с одним ведущим узлом в случае отказа ЦОДа, в котором находится ведущий узел, восстановление после сбоя сделает ведомый узел в другом ЦОДе ведущим. При схеме с несколькими ведущими узлами каждый ЦОД может работать независимо от остальных и репликация наверстывает упущенное после возобновления работы отказавшего ЦОДа.

Устойчивость к проблемам с сетью. Трафик между ЦОДами обычно проходит через общедоступный Интернет — менее надежный, чем локальная сеть внутри ЦОДа. Схема с одним ведущим узлом очень чувствительна к сбоям работы этого соединения между ЦОДами, поскольку операции записи выполняются через него синхронно. Схема асинхронной репликации с несколькими ведущими
узлами обычно более устойчива к проблемам с сетью: временные сбои работы сети не мешают обработке операций записи.

Некоторые базы данных поддерживают схему с несколькими ведущими узлами по умолчанию, но она также часто реализуется с помощью внешних утилит, например BDR для PostgreSQL.

Поскольку репликация с несколькими ведущими узлами — дополнительно доработанная возможность во многих базах данных, есть немало коварных подводных камней настроек и неожиданных взаимодействий с другими возможностями этих баз. Например, проблематичным может оказаться использование автоматически увеличиваемых ключей, триггеров и ограничений целостности. Поэтому репликация с несколькими ведущими узлами часто считается опасной вещью, которой лучше избегать по мере возможности.

### Офлайн-клиенты

Другая ситуация, подходящая для использования репликации с несколькими ведущими узлами, — приложения, которые должны продолжать работать, даже будучи отключенными от Интернета.


Например, рассмотрим приложения-календари на мобильном телефоне, ноутбуке и других устройствах. У вас должна сохраняться возможность в любой момент просмотреть расписание ваших совещаний (выполнять запросы на чтение) и вводить в него новые совещания (выполнять запросы на запись) независимо от наличия соединения с Интернетом. Любые выполняемые в офлайн-режиме изменения должны синхронизироваться с сервером и остальными вашими устройствами при следующем подключении к Интернету.

В этом случае у каждого устройства есть своя локальная база данных, служащая ведущим узлом (она принимает запросы на запись). Кроме того, происходит асинхронный процесс репликации (синхронизации) между репликами календаря на всех устройствах. Задержка репликации может составлять часы или даже дни в зависимости от того, когда появится доступ в Интернет.

С точки зрения архитектуры такая схема, по сути, то же самое, что и репликация с несколькими ведущими узлами между ЦОДами, доведенная до крайности: каждое устройство представляет собой ЦОД, а сетевое соединение между ними исключительно ненадежно.

### Совместное редактирование

Приложения для совместного редактирования в режиме реального времени (realtime collaborative editing) предоставляют возможность нескольким людям редактировать документ одновременно. Например, Google Docs позволяет одновременно редактировать документ или электронную таблицу.

Обычно совместное редактирование не рассматривают как задачу репликации базы данных, но с ранее упомянутым офлайн-редактированием у него много общего. Когда пользователь редактирует документ, изменения немедленно применяются к его локальной реплике (состоянию документа в его браузере или клиентском приложении) и асинхронно реплицируются на сервер и на всех остальных пользователей, редактирующих этот же документ.

При необходимости обеспечить отсутствие конфликтов редактирования приложение должно, прежде чем пользователь сможет отредактировать документ, запросить на этот документ блокировку. Если другой пользователь хочет отредактировать тот же документ, он должен сначала подождать, пока первый не зафиксирует свои изменения и не снимет блокировку. Такая модель совместной работы эквивалентна репликации с одним ведущим узлом и выполнением транзакций на ведущем узле.

Однако для ускорения совместной работы желательно сделать блок изменений очень маленьким (например, одно нажатие клавиши) и по возможности избегать блокировок. Такой подход позволяет редактировать нескольким пользователям одновременно, но также влечет за собой все проблемы репликации с несколькими ведущими узлами, включая необходимость разрешения конфликтов.

### Сходимость к согласованному состоянию

База данных с одним ведущим узлом применяет операции записи в последовательном порядке: при наличии нескольких изменений одного поля последняя по времени операция записи определяет итоговое значение этого поля.

В схеме с несколькими ведущими узлами нельзя задать порядок операций записи, так что непонятно, каким должно быть итоговое значение. Если в одном ведущем узле 1 название A сначала меняется на B, а позднее на C, а ведущем узле 2 оно сначала меняется на C, а потом — на B, то никакой из этих порядков нельзя назвать «более правильным, чем другой».

Если все реплики просто будут применять операции записи в том порядке, в котором они эти операции записи получают, то база данных в конце концов окажется в несогласованном состоянии: итоговое значение будет C в ведущем узле 1 и B — в ведущем узле 2. Это недопустимо: каждая схема репликации обязана обеспечить наличие одинаковых данных во всех репликах. Следовательно, база должна разрешать конфликт конвергентным способом, то есть все реплики должны сойтись к одному значению после репликации всех изменений.

Существует множество способов конвергентного разрешения конфликтов.

Присвоить каждой операции записи уникальный идентификатор (например, метку даты/времени, случайное длинное число, UUID или хеш ключа и значения), после чего просто выбрать операцию («победителя») с максимальным значением этого идентификатора, а остальные отбросить. В случае использования метки даты/времени в качестве идентификатора этот метод известен под названием «выигрывает последний» (last write wins, LWW). Хотя этот подход очень популярен, ему свойственно терять данные [35].

Присвоить уникальный идентификатор каждой реплике и считать, что у исходящих от реплик с большим номером операций записи есть приоритет перед теми, которые исходят от реплик с меньшим. Этот подход также приводит к потерям данных.

Заносить конфликты в заданную в явном виде структуру данных для хранения и написать код приложения, который бы разрешал конфликты позднее (возможно, спрашивая для этого пользователя).

### Секционирование БД

Секционирование (или шардинг) - в отличие от репликации, при секционировании происходит разбиение большой базы данных на небольшие подмножества, называемые секциями (partitions), в результате чего разным узлам можно поставить в соответствие различные секции.

Репликация и секционирование не противоречат другу, их можно применять одновременно. Более того, секционирование обычно идет бок о бок с репликацией, вследствие чего копии
каждой из секций хранятся на нескольких узлах. Это значит, что, хотя каждая конкретная запись относится только к одной секции, храниться она может в нескольких различных узлах в целях отказоустойчивости.

Обычно секции задаются таким образом, что каждый элемент данных (запись, строка или документ) относятся ровно к одной секции. Этого можно достичь множеством способов, часть из которых мы обсудим подробно ниже. Фактически каждая секция сама по себе является маленькой БД, хотя база способна поддерживать операции, затрагивающие сразу несколько секций.

Основная цель секционирования данных — масштабируемость. Разные секции можно разместить в различных узлах в кластере, не предусматривающем разделения ресурсов. Следовательно, большой набор данных можно распределить по многим жестким дискам, а запросы — по многим процессорам.

### Секционирование по диапазонам значений ключа

Один из методов секционирования — назначить каждой из секций непрерывный диапазон значений ключа (от какого-то минимального значения до какого-то максимального), подобно томам бумажной энциклопедии. Если вам известны границы между диапазонами, то можно легко определить, в какой секции содержится нужный ключ. Если вдобавок знать, с какими секциями какие узлы соотносятся, то можно выполнить запрос непосредственно к соответствующему узлу (или — в случае энциклопедии — снять требуемую книгу с полки).

Однако недостатком секционирования по диапазонам значений ключа является то, что некоторые паттерны доступа приводят к возникновению горячих точек (hot spot, секции с непропорционально высокой нагрузкой). Если ключ представляет собой метку даты/времени, то секции соответствуют отрезкам времени, например одна секция на день. К сожалению, в случаях, когда данные записываются в базу по мере получения значений от датчиков, все операции записи будут выполняться в одной секции (сегодняшней), вследствие чего эта секция окажется перегружена операциями записи, а остальные будут простаивать.

Чтобы избежать этой проблемы в базе данных, полученных с датчиков, необходимо использовать в качестве первого элемента ключа не метку даты/времени, а нечто иное. Например, можно предварить метку даты/времени названием датчика, и, как следствие, секционирование будет выполняться сначала по названию датчика, а только потом по времени. При условии одновременной работы множества датчиков нагрузка по записи распределится по секциям более равномерно. Но при необходимости получить значения с нескольких датчиков в пределах определенного промежутка времени придется выполнять отдельный запрос по диапазону для каждого из названий датчиков.

### Секционирование по хешу ключа

Многие распределенные базы данных, опасаясь асимметрии и горячих точек, используют для распределения ключей по секциям хеш-функцию. Хорошая хеш-функция получает на входе асимметричные данные и возвращает равномерно распределенные значения. Допустим, у вас есть 32-битная хешфункция, получающая на входе строковое значение. Для каждой строки на входе она возвращает псевдослучайное число в диапазоне от 0 до 232 – 1. Даже если входные строки очень похожи, их хеши равномерно распределены по этому интервалу. Например, СУБД Cassandra и MongoDB используют MD5.

При наличии подходящей хеш-функции для ключей можно поставить каждой секции в соответствие диапазон хешей (вместо диапазона ключей), и каждый ключ, чье значение находится в диапазоне данной секции, будет сохранен в ней.

Однако, к сожалению, при использовании для секционирования хеша ключа мы теряем удобное свойство секционирования по диапазонам значений ключа: возможность эффективно выполнять запросы по диапазонам. Смежные некогда ключи оказываются разбросаны по всем секциям, и порядок их сортировки теряется. В MongoDB при включении режима шардинга на основе хеша любой запрос по диапазону приходится отправлять все секции.

### Асимметричные нагрузки и разгрузка горячих точек

Как уже обсуждалось выше, хеширование ключа для определения секции может несколько исправить ситуацию с горячими точками, но не избавиться от них полностью: в предельном случае, когда все операции записи и чтения выполняются для одного ключа, все запросы все равно приходятся на одну секцию.

Подобная нагрузка, вероятно, необычна, но не беспрецедентна: например, на сайтах соцсетей действия знаменитостей, насчитывающих миллионы подписчиков, могут вызывать бурю активности. Такие события могут привести к огромным объемам операций записи для одного ключа (где ключом, вероятно, будет ID пользователя этой знаменитости или ID действия, которое комментируют подписчики). Хеширование ключа не поможет в такой ситуации, ведь хеш двух одинаковых ключей — одинаковый.

На сегодняшний день большинство информационных систем не умеют автоматически выравнивать подобную высоко асимметричную нагрузку, вследствие чего снижение асимметрии — обязанность приложения. Например, если известно, что конкретный ключ — очень горячий, то простейшим решением будет добавление в начало или конец этого ключа случайного числа. Простое двузначное десятичное число приведет к разбиению операций записи для ключа равномерно по 100 различным ключам, что позволит распределить их по разным секциям.

Однако при условии разбиения операций записи по различным ключам операциям чтения придется совершать дополнительные действия по чтению и объединению данных для всех этих ключей. Описанный метод требует также дополнительных вспомогательных операций: добавлять случайное число имеет смысл только для небольшого числа горячих ключей, для абсолютного большинства ключей с низкими объемами операций записи это окажется лишними накладными расходами. Следовательно, понадобится отслеживать, какие ключи были разбиты.

Возможно, в будущем информационные системы смогут автоматически обнаруживать и выравнивать асимметричную нагрузку, но пока что вам придется самим обдумывать плюсы и минусы подобного подхода для ваших приложений.

### NoSQL

Перед тем, как углубится в, собственно, Postgres, пройдёмся немного по классу СУБД, объединенных понятием [NoSQL](https://en.wikipedia.org/wiki/NoSQL).

Так же, как рост размера файлов данных предопределил переход от файл-серверных БД к клиент-серверным, так и рост требований к масштабируемости и доступности вызвал появление и развитие NoSQL БД.

Нереляционные хранилища данных NoSQL делятся на две основные разновидности:  
1. Документоориентированные БД предназначены для тех сценариев использования, при которых данные поступают в виде отдельных документов и связи между документами минимальны.  
2. Графовые БД, наоборот, предназначены для сценариев применения, в которых любые данные потенциально могут быть связаны между собой.

Все три модели (реляционная, документная и графовая) в настоящее время широко используются, и все отлично работают в своих предметных областях. Одну модель можно эмулировать на языке другой (например, графовые данные разместить в реляционной БД), но результат этого часто достаточно неудобен в применении. Именно поэтому для разных целей прибегают к разным системам, а не к единому универсальному решению.

Одна из общих черт документоориентированных и графовых БД — отсутствие обязательной схемы для хранимых данных, что облегчает адаптацию приложений к изменению требований. Однако чаще всего приложение предполагает наличие какой-либо структуры данных, вопрос только в том, является ли схема явной (навязываемой при записи) или неявной (контролируемой при чтении).

Идея столбцовых хранилищ проста: нужно хранить рядом значения не из одной строки, а из одного столбца, причём каждый столбец хранится в отдельном файле.  

Размещение данных по столбцам требует, чтобы файлы всех столбцов содержали строки в одинаковом порядке. Следовательно, при необходимости собрать воедино целую строку можно взять из всех файлов отдельных столбцов 20-й элемент и собрать их вместе для формирования 20-й строки таблицы.

Столбцовое хранилище часто очень хорошо поддается сжатию.

OLTP-системы обычно нацелены на работу с пользователями, и это означает огромное потенциальное количество запросов. Чтобы справиться с такой нагрузкой, приложения обычно затрагивают в каждом запросе только небольшое число строк. Программы запрашивают записи с помощью определенного ключа, а подсистема хранения задействует индекс для поиска данных с соответствующим ключом. Узким местом здесь обычно становится время перехода к нужной позиции на диске. Склады данных и подобные им аналитические системы менее широко известны, поскольку они в основном применяются бизнес-аналитиками, а не конечными пользователями. Склады обрабатывают намного меньшее количество запросов, чем OLTP-системы, но все запросы обычно очень ресурсоемки и требуют просмотра миллионов строк за короткое время. Узким место здесь обычно становится пропускная способность диска (а не время перехода к нужной позиции на нем). Все большую популярность для этой разновидности задач приобретают столбцовые хранилища.

Не все склады данных являются столбцовыми хранилищами: используются и обычные построчные БД, а также несколько других архитектур. Однако столбцовые хранилища работают намного быстрее при произвольных аналитических запросах, так что их популярность быстро растет.

Подумать над переходом на NoSQL стоит, если:  
ваше приложение нуждается в экстремально низкой латентности;  
вы работаете с неструктурированными данными, не имеющих реляционных связей;  
вам нужно всего лишь сериализовать и десериализовать свои данные (JSON, XML и т. д.);  
вам нужно работать с очень большим объемом данных.

Две, пожалуй, основные черты всех NoSQL решений — **отказ от согласованности данных** и относительно лёгкая **масштабируемость**.

В качестве обоснования неизбежности отказа от согласованности данных используется т. н. [теорема CAP](https://en.wikipedia.org/wiki/CAP_theorem), утверждающая, что в любой реализации распределённых вычислений возможно обеспечить не более двух из трёх следующих свойств:  
• согласованность данных (consistency) — во всех вычислительных узлах в один момент времени данные не противоречат друг другу;  
• доступность (availability) — любой запрос к распределённой системе завершается корректным откликом, однако без гарантии, что ответы всех узлов системы совпадают;  
• устойчивость к разделению (partition tolerance) — расщепление распределённой системы на несколько изолированных секций не приводит к некорректности отклика от каждой из секций.

В качестве противопоставления требованиям ACID, рассмотренным выше, для NoSQL сформулированы требования BASE:  
• базовая доступность (basic availability) — подход к проектированию, требующий, чтобы сбой в некоторой части узлов приводил к отказу в обслуживании только для части пользователей при сохранении доступности в большинстве случаев;  
• гибкое состояние (soft state) — возможность жертвовать хранением состояния сессий (промежуточные результаты, информация о контексте), концентрируясь на фиксации обновлений только критичных операций;  
• согласованность в конечном счёте (eventual consistency) — данные могут быть некоторое время рассогласованы, при обеспечении согласования в практически обозримое время.

NoSQL решения были широко распространены примерно до 1970-х, когда на сцене появились реляционные БД. Повторный расцвет популярности NoSQL пришелся на 2010-е, когда Google на примере BigTable показал их эффективность применительно к некоторым типам задач.

Порой выбор между SQL и NoSQL нелегок и требует, конечно, большого опыта работы с БД, у обоих принципов есть свои преимущества и недостатки. В целом, кандидатами на применение NoSQL решений являются системы, объединяющие большое количество данных и некритичность к возможности доступа каждого пользователя к самой последней версии данных. Поисковая система (Google), крупный интернет-магазин (Amazon) — хорошие кандидаты на применение NoSQL. Напротив, банковская или биржевая система, в рамках которой каждому пользователю должны предоставляться самые свежие, актуальные данные требуют применения SQL решений.

### Реляционная модель данных

PostgreSQL является **реляционной** СУБД (РСУБД). Теоретической основой для работы с реляционными базами данных служит раздел математики под названием [реляционная алгебра](https://en.wikipedia.org/wiki/Relational_algebra). [Реляционная модель данных](https://en.wikipedia.org/wiki/Relational_model) (РМД) основана на математическом понятии «отношения» (relation), которое в практическом смысле, на уровне программиста БД, можно толковать как «таблица». Соответственно, реляционную модель данных можно упрощенно воспринимать как «табличную модель данных», т. е. построенную на основе двумерных таблиц, состоящих из строк и столбцов.

Вот небольшой словарик для «перевода» понятий реляционной алгебры на более простой практический «табличный» язык, который может вам пригодиться при чтении обучающей литературы:  
сущность — объекты, содержащиеся в БД (например, сотрудники, клиенты, заказы);  
отношение — таблица;  
атрибут — столбец;  
кортеж — строка (или запись);  
результирующий набор — результат SQL-запроса (как правило, это таблица; в частном случае — таблица из одной строки и одного столбца, т. е., например, число или строка).

Работая с реляционной БД, программисту не нужно заботиться о низкоуровневом доступе к данным, достаточно описать, *что* нужно получить, а *как* именно — описывать не нужно, эту работу берет на себя БД.

Отсутствие низкоуровневого доступа — сильное ограничение, порождающее удобство использования БД (о плюсах ограничений мы поговорим чуть позже, в разделе «Архитектура»). Но это ограничение буквально «ломает мозг» программисту, не имевшему до этого дела с [декларативными](https://en.wikipedia.org/wiki/Declarative_programming) языками программирования.

Когда программист, работающий в императивной парадигме, пытается решить SQL-задачу, руки сами тянутся применить привычный набор инструментов. «Так, проходимся по таблице в цикле, отсеянные значения складываем во временную переменную, потом формируем ответ», но — SQL предлагает совсем другой, непривычный набор возможностей, не предполагая прямого управления ходом выполнения программы: «Скажи, что ты хочешь, а о деталях я позабочусь сам».

Если такой подход для вас внове, не отчаивайтесь; Джо Селко в своей книге «Thinking in Sets» сразу рекомендует готовиться к длительной перестройке процесса мышления: «Я ориентирую студентов на срок около года, полностью посвященного программированию на SQL, только тогда вас может посетить прозрение и вы начнете думать на языке SQL» («I have been telling students that you need about one year of full-time SQL programming before you have an epiphany and start thinking in SQL»).

Так что (продолжая цитировать книгу Селко) потихоньку избавляйтесь от своего «procedural programming mindset», который приводит к «overly complex and inefficient code», и «change the way you think about the problems you solve with SQL programs». Или, по простому, как советуют пользователи Stackoverflow в этом [топике](https://stackoverflow.com/questions/1119505/how-to-think-in-sql), «прекратите думать о построчной обработке, подумайте над решением задачи, основанном на работе со множествами».

### Язык SQL

SQL (structured query language, язык структурированных запросов) — декларативный (описательный, непроцедурный) язык, стандарт для работы с данными во всех реляционных СУБД.  
Операторы SQL традиционно делят на:  
операторы определения данных (data definition language, **DDL**), например, CREATE, ALTER, DROP;  
операторы манипулирования данными (data manipulation language, **DML**), таких как SELECT, INSERT, DELETE;  
операторы управления транзакциями (transaction control language, **TCL**), например, COMMIT, ROLLBACK;  
операторы управления привилегиями доступа (data control language, **DCL**), к примеру, GRANT, REVOKE, DENY.

### Многообразие диалектов СУБД. ANSI SQL.

Прежде чем начать, наконец-то, писать запросы к БД, нам нужно определиться с конкретной СУБД, которую мы будем использовать в дальнейшем (спойлер — как и следует из названия главы, «Базы данных (с уклоном в PostgreSQL)», это будет Postgres). К сожалению, диалекты языка SQL для разных СУБД достаточно сильно различаются и писать универсальные запросы, подходящие ко всем популярным СУБД, у нас не получится, волей-неволей придётся сосредоточиться на одной конкретной реализации.

Общий знаменатель у разных диалектов есть, называется он ANSI SQL и имеет несколько актуальных редакций, от [SQL-92](https://en.wikipedia.org/wiki/SQL-92) до [SQL-2023](https://en.wikipedia.org/wiki/SQL:2023). SQL-92, несмотря на дату публикации, до сих пор вполне актуален и изучение ANSI SQL стоит начать именно с него. К сожалению, ANSI SQL выступает скорее в качестве своеобразного маяка, определяющего разработку СУБД в целом, но не накладывающего жестких ограничений на синтаксис языка.

Существуют попытки создать обучающие руководства на базе ANSI SQL, например, [Wikibook SQL](https://en.wikibooks.org/wiki/Structured_Query_Language), но при практическом изучении лучше сразу ориентироваться на конкретный диалект. ANSI SQL покрывает примерно 90 % возможностей каждого конкретного диалекта, что вроде бы выглядит как достаточно солидный базис, однако в практических сценариях желание использовать для обучения и работы ANSI SQL быстро исчезает, т. к. разница между диалектами слишком велика и попытка решения реальных задач на ANSI SQL приводит к постоянным поискам компромисса и неиспользованию возможностей вашего инструмента.

Учитывая сильное расхождение между диалектами и то, что часть возможностей ANSI SQL не реализована ни в одном из существующих диалектов (как сетуют разработчики Wikibook SQL, 18 мегабайт спецификации на 1400 страницах — не фунт изюму), для реальной работы придется выбрать какой-то один диалект SQL. Но это не такая большая проблема, как кажется, и после изучения одного из диалектов остальные варианты дадутся вам уже гораздо проще.

### Выбор СУБД. SQLite, MySQL, PostgreSQL, SQL Server, Oracle SQL.

Как я уже говорил, в дальнейшем мы будем использовать PostgreSQL. Прежде чем окончательно перейти на Postgres, уделим немного внимания его конкурентам.

### SQLite

Small. Fast. Reliable. Choose any three.  

https://sqlite.org/index.html  
https://github.com/sqlite/sqlite  
https://habr.com/ru/post/149356/  
https://github.com/sqlitebrowser/sqlitebrowser  

Syntax Diagrams https://www.sqlite.org/syntaxdiagrams.html  

Server-less database engine that stores each database into a separate file.

### MySQL

### SQL Server

### Oracle SQL

Перейдём к нашему рабочему инструменту, PostgreSQL.


### Преимущества PostgreSQL

[Каскадные триггеры](https://www.postgresql.org/docs/current/sql-createtrigger.html). Если триггерная функция выполняет команды SQL, эти команды могут заново запускать триггеры. Насколько я знаю, каскадные триггеры есть и в SQL Server.

[hstore](https://www.postgresql.org/docs/current/hstore.html). Возможность создавать и манипулировать данными с функциональностью словаря (dictionary).

[JSONB](https://www.postgresql.org/docs/current/datatype-json.html). Парсинг JSON осуществляется однократно, во время записи. Более медленная однократная запись, но более быстрые многократные чтения. По умолчанию рекомендуется использовать JSONB, а не JSON.

[Range types](https://www.postgresql.org/docs/current/rangetypes.html). Никаких больше колонок planned_worktime_start и planned_worktime_end и пляски с операторами сравнения для нахождения других строк, у которых интервал, задаваемый этими колонками пересекается с этой строкой.

Другие удобные нативные типы: [interval](https://www.postgresql.org/docs/current/functions-datetime.html), [macaddr](https://www.postgresql.org/docs/current/datatype-net-types.html#DATATYPE-MACADDR) и [другие](https://www.postgresql.org/docs/current/datatype.html), со встроенными методами работы с ними.

[Массивы](https://www.postgresql.org/docs/current/arrays.html) — нарушение 1-й нормальной формы, но когда всё, что необходимо — это сохранить несколько строчек, то добавление отдельной таблицы с перспективой JOIN'а с ней выглядит совсем непривлекательно.

У PostgreSQL полностью транзакционный DDL, т.е. можно в транзакциях менять схему данных, и эти изменения буду транзакционными. Соответственно, возможны миграции без остановки записи.

[PostGIS](https://postgis.net/). Бесплатное расширение для PostgreSQL с открытым исходным кодом для работы с географическими объектами, дополняющее встроенные возможности БД (point, gist). Работает с точками, ломаными линиями, полигонами, растрами, а также использует их для разных операций, например, поиска.

[PL/pgSQL](https://www.postgresql.org/docs/9.6/plpgsql.html). Процедурный язык для PostgreSQL. Функции PL/pgSQL могут использоваться везде, где допустимы встроенные функции. Например, можно создать функции со сложными вычислениями и условной логикой, а затем использовать их при определении операторов или в индексных выражениях.

### Типы данных PostgreSQL

**Целые числа**

smallint 2 байта  
integer 4 байта Самый распространенный целочисленный тип  
bigint 8 байт

**Автоинкрементируемые целые числа**

smallserial 2 байта  
serial 4 байта  
bigserial 8 байт

Фактически, serial — не истинный тип, а "синтаксический сахар" для замены группы SQL-команд.

Синтаксис для создания столбца типа serial таков:

```sql
CREATE TABLE имя-таблицы (имя-столбца serial);
```

Эта команда эквивалентна следующей группе команд:

```sql
CREATE SEQUENCE имя-таблицы_имя-столбца_seq;
CREATE TABLE имя-таблицы
(
имя-столбца integer NOT NULL
DEFAULT nextval('имя-таблицы_имя-столбца_seq')
);
ALTER SEQUENCE имя-таблицы_имя-столбца_seq
OWNED BY имя-таблицы.имя-столбца;
```

Здесь используется функция nextval, которая получает очередное число из последовательности (по сути, из генератора уникальных целых чисел).

**Вещественные числа**

float4 (real) 4 байта 6 знаков после запятой Неточное значение, накопление ошибок при проведении арифметических операций  
float8 (float, double precision) 8 байт 15 знаков после запятой Неточное значение  

Типы данных real и double precision поддерживают специальное значение NaN, которое означает «не число» (not a number). В математике существует такое понятие, как неопределенность. В качестве одного из ее вариантов служит результат операции умножения нуля на бесконечность. Посмотрите, что выдаст в результате PostgreSQL:
SELECT 0.0 * 'Inf'::real;

NaN
(1 строка)

Значение NaN считается равным другому значению NaN, а также что значение NaN считается большим любого другого «нормального» значения, т. е. не-NaN.

Например, можно сравнить значения NaN и Infinity.

```sql
SELECT 'NaN'::real > 'Inf'::real;
```

decimal (numeric) Точное значение. Денежные рассчеты

**Символы**

char строка фиксированной длины, дополняется пробелами  
varchar изменяемой длины с лимитом  
text нелимитируемой длины

Документация рекомендует использовать типы text и varchar, поскольку такое отличительное свойство типа character, как дополнение значений пробелами, на практике почти не востребовано. В PostgreSQL обычно используется тип text.

Для экранирования символов, например, одинарной кавычки или обратной косой черты, их удваивают. Если экранируемых символов слишком много и это затрудняет чтение текста, строку можно одинарных ковычек заключить в двойные символы «$».

PostgreSQL предлагает еще одно расширение стандарта SQL — строковые константы в стиле языка C. Чтобы иметь возможность их использовать, нужно перед начальной одинарной кавычкой написать символ «E». Например, для включения в константу символа новой строки нужно сделать так:

```sql
SELECT E'Hello\nworld!';
```

**Логическое значение**

Boolean (bool) 1 байт

В качестве истинного состояния могут служить следующие значения: TRUE, 't', 'true', 'y', 'yes', 'on', '1'.  
В качестве ложного состояния могут служить следующие значения: FALSE, 'f', 'false', 'n', 'no', 'off', '0'.

При работе с логическими значениями в условии WHERE можно не писать выражение WHERE is_value = 'yes', а достаточно просто указать WHERE is_value.

**Представление даты и времени**

date. По умолчанию используется формат, рекомендуемый стандартом ISO 8601: «yyyy-mm-dd».

Чтобы «сказать» СУБД, что введенное значение является датой, а не простой символьной строкой, нужно использовать операцию приведения типа. В PostgreSQL она оформляется с использованием двойного символа двоеточия и имени того типа, к которому мы приводим данное значение. Важно учесть, что при выполнении приведения типа производится проверка значения на соответствие формату целевого типа и множеству его допустимых значений.

```sql
SELECT 'Sep 01, 2023'::date;
```

Можно использовать команду получения текущей даты:

```sql
SELECT current_date;
```

time  
time with time zone  

Время вводить как в 12-часовом, так и в 24-часовом формате:

```sql
SELECT '22:15:10'::time;

SELECT '10:15:10 pm'::time;
```

Текущее время:

```sql
SELECT current_time;
```

Типы timestamp, time и interval позволяют задать точность ввода и вывода значений. Точность предписывает количество десятичных цифр в поле секунд. Проиллюстрируем эту возможность на примере типа time, выполнив три запроса: в первом запросе вообще не используем параметр точности, во втором назначим его равным 0, в третьем запросе сделаем его равным 3.

SELECT current_time;

timetz
--------------------

19:46:14.584641+03
(1 строка)
SELECT current_time::time( 0 );

time
----------

19:39:45
(1 строка)
SELECT current_time::time( 3 );

time
--------------

19:39:54.085
(1 строка)

timestamp. временная метка — объединение типов даты и времени.  
timestamp with time zone (timestamptz)

Получение текущей временной метки:

```sql
SELECT current_timestamp;
```

Значения временных меток можно округлять с помощью функции date_trunc. Например, для получения текущей временной отметки с точностью до одного часа нужно сделать так:

```sql
SELECT (date_trunc('hour', current_timestamp));
```

Из значений временных отметок можно с помощью функции extract извлекать отдельные поля, т. е. год, месяц, день, число часов, минут или секунд и т. д. Например, чтобы извлечь номер месяца, нужно сделать так:

```sql
SELECT extract('mon' FROM timestamp '2023-11-21 12:35:555.123456');
```

interval Разница между двумя timestamp

Например:

```sql
SELECT '1 year 6 months ago'::interval;
```

или

```sql
SELECT ('2021-08-16'::timestamp - '2021-08-01'::timestamp)::interval;
```

Формат ввода и вывода даты можно изменить с помощью конфигурируемого параметра datestyle. Значение этого параметра состоит из двух компонентов: первый управляет форматом вывода даты, а второй регулирует порядок
следования составных частей даты (год, месяц, день) при вводе и выводе. Текущее значение этого параметра можно узнать с помощью команды SHOW:

SHOW datestyle;

**Массивы**

PostgreSQL позволяет создавать в таблицах такие столбцы, в которых будут содержаться не отдельные значения, а массивы переменной длины. Эти массивы могут быть многомерными и могут содержать значения любого из встроенных типов, а также типов данных, определенных пользователем.

Для указания на то, что это массив, нужно добавить квадратные скобки к наименованию типа данных, при этом задавать число элементов не обязательно.

Пример:

```sql
CREATE TABLE gamers
(
name text,
points integer[]
);

INSERT INTO gamers
VALUES ( 'Ivan', '{ 1, 3, 5, 6, 7 }'::integer[] ),
       ( 'Mike', '{ 1, 2, 5, 7 }'::integer[] ),
       ( 'Jake', '{ 2, 5 }'::integer[] ),
       ( 'Finn', '{ 3, 5, 6 }'::integer[] )
```

Добавление значения в массив:

```sql
UPDATE gamers
SET points = points || 8
WHERE name = 'Jake';
```

**json, jsonb**

Типы JSON предназначены для сохранения в столбцах таблиц базы данных таких значений, которые представлены в формате JSON (JavaScript Object Notation). Существует два типа: json и jsonb. Основное различие между ними заключается в быстродействии. Если столбец имеет тип json, тогда сохранение значений происходит быстрее, потому что они записываются в том виде, в котором были введены. Но при последующем использовании этих значений в качестве операндов или параметров функций будет каждый раз выполняться их разбор, что замедляет работу. При использовании типа jsonb разбор производится однократно, при записи значения в таблицу. Это
несколько замедляет операции вставки строк, в которых содержатся значения данного типа. Но все последующие обращения к сохраненным значениям выполняются быстрее, т. к. выполнять их разбор уже не требуется.

Есть еще ряд отличий, в частности, тип json сохраняет порядок следования ключей в объектах и повторяющиеся значения ключей, а тип jsonb этого не делает. Рекомендуется в приложениях использовать тип jsonb, если только нет каких-то особых аргументов в пользу выбора типа json.

**NULL — отсутствие данных**


Есть также XML, геометрические типы, типы, определяемые пользователем и прочее.

### DDL

### Constraints (ограничения)

```sql
column_constraint is:

[ CONSTRAINT constraint_name ]
{ NOT NULL |
  NULL |
  CHECK ( expression ) [ NO INHERIT ] |
  DEFAULT default_expr |
  GENERATED ALWAYS AS ( generation_expr ) STORED |
  GENERATED { ALWAYS | BY DEFAULT } AS IDENTITY [ ( sequence_options ) ] |
  UNIQUE [ NULLS [ NOT ] DISTINCT ] index_parameters |
  PRIMARY KEY index_parameters |
  REFERENCES reftable [ ( refcolumn ) ] [ MATCH FULL | MATCH PARTIAL | MATCH SIMPLE ]
    [ ON DELETE referential_action ] [ ON UPDATE referential_action ] }
[ DEFERRABLE | NOT DEFERRABLE ] [ INITIALLY DEFERRED | INITIALLY IMMEDIATE ]

and table_constraint is:

[ CONSTRAINT constraint_name ]
{ CHECK ( expression ) [ NO INHERIT ] |
  UNIQUE [ NULLS [ NOT ] DISTINCT ] ( column_name [, ... ] ) index_parameters |
  PRIMARY KEY ( column_name [, ... ] ) index_parameters |
  EXCLUDE [ USING index_method ] ( exclude_element WITH operator [, ... ] ) index_parameters [ WHERE ( predicate ) ] |
  FOREIGN KEY ( column_name [, ... ] ) REFERENCES reftable [ ( refcolumn [, ... ] ) ]
    [ MATCH FULL | MATCH PARTIAL | MATCH SIMPLE ] [ ON DELETE referential_action ] [ ON UPDATE referential_action ] }
[ DEFERRABLE | NOT DEFERRABLE ] [ INITIALLY DEFERRED | INITIALLY IMMEDIATE ]
```

Constraints указываются при создании или изменении таблицы. Эти правила ограничивают тип данных, которые могут храниться в таблице. При нарушении ограничений действия с данными выполнены не будут.

NOT NULL — значение не может быть NULL;

CHECK — значения столбца должны соответствовать заданным условиям.

DEFAULT — назначает столбцу значение по умолчанию.

UNIQUE — гарантирует уникальность значений в столбце.

PRIMARY KEY

внешний ключ (foreign key)

Exclusion Constraints

### CREATE, создание БД и таблиц

Создание новой БД:

```sql
CREATE DATABASE testdb
    WITH
    OWNER = postgres
    ENCODING = 'UTF8';
```

Создание новых таблиц:
```sql
CREATE TABLE publisher
(
    publisher_id integer PRIMARY KEY,
    org_name varchar(128) NOT NULL,
    address text NOT NULL
);

CREATE TABLE book
(
    book_id integer PRIMARY KEY,
    title text NOT NULL,
    isbn varchar(32) NOT NULL
)
```

PRIMARY KEY (первичный ключ) предназначен для однозначной идентификации каждой записи в таблице и является строго уникальным, две записи таблицы не могут иметь одинаковые значения первичного ключа. Нулевые значения (NULL) в PRIMARY KEY не допускаются. Если в качестве PRIMARY KEY используется несколько полей, их называют составным ключом.

### DROP, удаление таблицы

```sql
DROP TABLE publisher;
DROP TABLE book
```

### ALTER, изменение структуры таблицы

Добавляем колонку:
```sql
ALTER TABLE book
ADD COLUMN fk_publisher_id integer;
```

### FOREIGN KEY, внешний ключ

Сообщаем о том, что эта колонка будет внешним ключом, ссылающимся на publisher_id в другой таблице
```sql
ALTER TABLE book
ADD CONSTRAINT fk_book_publisher
FOREIGN KEY (fk_publisher_id) REFERENCES publisher(publisher_id)
```

Структура таблицы будет приведена к виду, как если бы мы создавали её при помощи следующих команд:
```sql
CREATE TABLE book
(
    book_id integer PRIMARY KEY,
    title text NOT NULL,
    isbn varchar(32) NOT NULL,
    fk_publisher_id integer REFERENCES publisher(publisher_id) NOT NULL
)
```

Мы создали отношение «один ко многим» (одно издательство может издавать множество книг), самое часто используемое отношение в SQL.

### RENAME

### TRUNCATE

### CREATE INDEX 

создаёт индексы в таблице для быстрого поиска/запросов;

Общая информация
Индексы по нескольким столбцам
Уникальные индексы
Индексы на основе выражений
Частичные индексы

Работа с индексами и ограничениями  

Индексы  
Индекс – специальная структура данных, которая связана с таблицей и создаётся на основе её данных. Индексы создаются для повышения производительности функционирования базы данных.

Какие бывают индексы?

В-дерево

хеш

GiST

SP-GiST

GIN

BRIN

По умолчанию команда CREATE INDEX создаёт индексы типа В-дерево (эффективны в большинстве случаев)

Как можно создать индексы?

• Индекс по столбцу (это чистая классика)

• Индекс по нескольким столбцам

• Уникальный индекс

• Индекс на основе выражения

• Частичный индекс

Для создания индекса используется примерно такой синтаксис:

CREATE [UNIQUE] INDEX <index_name> ON <table_name> ( <column_name>, ... ) [STATEMENT] ;
При этом:

для создания уникального индекса может использоваться слово UNIQUE

для создания выражения его записывают в скобках, например для создания выражения проверки индекса на нижний регистр можно написать так:

. . . ( lower( <column_name> ) ) ;
для создания частичного индекса после скобок запись продолжается, например для проверки на величину можно написать так:

. . . ( . . . ) WHERE <column_name> > 1000 ;

### CREATE VIEW, Представления

При работе с базами данных зачастую приходится многократно выполнять одни и те же запросы, которые могут быть весьма сложными и требовать обращения к нескольким таблицам. Чтобы избежать необходимости многократного формирования таких запросов, можно использовать так называемые представления (views). Если речь идет о выборке данных, то представления практически неотличимы от таблиц с точки зрения обращения к ним в командах SELECT.

Упрощенный синтаксис команды CREATE VIEW, предназначенной для создания представлений:

```sql
CREATE VIEW имя-представления [ ( имя-столбца [, ...] ) ]
AS запрос;
```

Полный синтаксис:

```sql
CREATE [ OR REPLACE ] [ TEMP | TEMPORARY ] [ RECURSIVE ] VIEW name [ ( column_name [, ...] ) ]
    [ WITH ( view_option_name [= view_option_value] [, ... ] ) ]
    AS query
    [ WITH [ CASCADED | LOCAL ] CHECK OPTION ]
```

В этой команде обязательными элементами являются имя представления и запрос к базе данных, который и формирует выборку из нее. Если список имен столбцов не указан, тогда их имена «вычисляются» (формируются) на основании текста запроса.

Давайте создадим простое представление. В главе 3 мы решали задачу: подсчитать количество мест в салонах для всех моделей самолетов с учетом класса обслуживания (бизнес-класс и экономический класс). Запрос был таким:

```sql
SELECT aircraft_code,
fare_conditions,
count( * )
FROM seats
GROUP BY aircraft_code, fare_conditions
ORDER BY aircraft_code, fare_conditions;
```

На его основе создадим представление и дадим ему имя, отражающее суть этого представления.

```sql
CREATE VIEW seats_by_fare_cond AS
SELECT aircraft_code,
fare_conditions,
count( * )
FROM seats
GROUP BY aircraft_code, fare_conditions
ORDER BY aircraft_code, fare_conditions;
```

Теперь мы можем вместо написания сложного первоначального запроса обращаться непосредственно к представлению, как будто это обычная таблица.

```sql
SELECT * FROM seats_by_fare_cond;
```

В отличие от таблиц, представления не содержат данных. При каждом обращении к представлению в команде SELECT данные выбираются из таблиц, на основе которых это представление создано.

### Материализованное представление

Упрощенный синтаксис команды CREATE MATERIALIZED VIEW, предназначенной для создания материализованных представлений, таков:

```sql
CREATE MATERIALIZED VIEW [ IF NOT EXISTS ] имя-мат-представления
[ ( имя-столбца [, ...] ) ]
AS запрос
[ WITH [ NO ] DATA ];
```

В момент выполнения команды создания материализованного представления оно заполняется данными, но только если в команде не было фразы WITH NO DATA. Если же она была включена в команду, тогда в момент своего создания представление остается пустым, а для заполнения его данными нужно использовать команду REFRESH MATERIALIZED VIEW.

Материализованное представление очень похоже на обычную таблицу. Однако оно отличается от таблицы тем, что не только сохраняет данные, но также запоминает запрос, с помощью которого эти данные были собраны.

### DML

### INSERT, добавление новых строк

```sql
Insert INTO book
VALUES
(1, 'The Diary of a Young Girl', '0199535566', 1),
(2, 'Pride and Prejudice', '9780307594006', 1),
(3, 'To Kill a Mockingbird', '0446310786', 2),
(4, 'The book of a Gutsy Women: Favorite Stories of Courage and Resilience', '1501178415', 2),
(5, 'War and Peace', '1788886526', 2);

Insert INTO publisher
VALUES
(1, 'Everyman''s Library', 'NY'),
(2, 'Oxford University Press', 'NY'),
(3, 'Grand Central Publishing', 'Washington'),
(4, 'Simon & Schuster', 'Chicago');
```

Проверка правильности добавленных строк (более подробно оператор SELECT будет рассмотрен ниже):
```sql
SELECT *
FROM book
```

и

```sql
SELECT *
FROM publisher
```

WHERE и HAVING могут использоваться в одном запросе, при этом необходимо учитывать порядок исполнения SQL запроса:  
1. FROM  
2. ON  
3. JOIN  
4. WHERE  
5. GROUP BY  
6. WITH CUBE / WITH ROLLUP  
7. HAVING  
8. Оконные функции  
9. SELECT  
10. DISTINCT  
11. UNION / UNION ALL / INTERSECT / EXCEPT  
12. ORDER BY  
13. TOP / LIMIT / OFFSET  

Сначала определяется таблица, из которой выбираются данные (FROM);  
затем из этой таблицы отбираются записи в соответствии с условием WHERE;  
выбранные данные агрегируются (GROUP BY);  
из агрегированных записей выбираются те, которые удовлетворяют условию после HAVING.

Только потом формируются данные результирующей выборки, как это указано после SELECT (вычисляются выражения, присваиваются имена и пр.). Результирующая выборка сортируется в соответствии с условием, указанным после ORDER BY.  

Знание порядка исполнения SQL запроса важно для того, чтобы понять, почему, например, внутри WHERE невозможно использовать имена выражений из SELECT; просто SELECT выполняется компилятором позже, чем WHERE, поэтому ему неизвестно, какое выражение там описано.

### SELECT, FROM, WHERE

Команда SELECT получает записи из базы данных по определенному условию, которое задается с помощью команды WHERE.

Синтаксис:  
SELECT * FROM имя_таблицы;  
SELECT * FROM имя_таблицы WHERE условие;  
SELECT поле1, поле2... FROM имя_таблицы WHERE условие;  

Полный синтаксис команды SELECT:  
SELECT  
    [STRAIGHT_JOIN] [SQL_SMALL_RESULT] [SQL_BIG_RESULT] [SQL_BUFFER_RESULT]  
    [SQL_CACHE | SQL_NO_CACHE] [SQL_CALC_FOUND_ROWS] [HIGH_PRIORITY]  
    [DISTINCT | DISTINCTROW | ALL]  
select_expression,...  
[INTO {OUTFILE | DUMPFILE} 'file_name' export_options]  
[FROM table_references  
[WHERE where_definition]  
[GROUP BY {unsigned_integer | col_name | formula} [ASC | DESC], ...]  
[HAVING where_definition]  
[ORDER BY {unsigned_integer | col_name | formula} [ASC | DESC], ...]  
[LIMIT [offset,] rows | rows OFFSET offset]  
[PROCEDURE procedure_name(argument_list)]  
[FOR UPDATE | LOCK IN SHARE MODE]]  

### LIKE

### ORDER BY

Сортировка данных в порядке возрастания (ASC) или убывания (DESC).

```
SELECT * FROM user ORDER BY name DESC;
```

### COUNT, SUM, AVG, MAX, MIN, агрегирующие функции

### GROUP BY, HAVING, группировка и фильтрация данных

### JOIN, объединение данных из нескольких таблиц

#### INNER JOIN

Если совсем кратко, то INNER JOIN не пересечение множеств, как нарисовано, а, скорее, перемножение с условием.

#### FULL OUTER JOIN

#### LEFT JOIN

#### RIGHT JOIN

#### CROSS JOIN, перекрестное соединение

Создает набор строк, где каждая строка из одной таблицы соединяется с каждой строкой из второй таблицы.

Self join

Такой вопрос тоже может прозвучать на собеседовании по SQL. Это выражение используется для того, чтобы таблица объединилась сама с собой, словно это две разные таблицы. Чтобы такое реализовать, одна из таких «таблиц» временно переименовывается.

Например, следующий SQL-запрос объединяет клиентов из одного города:

```
SELECT A.CustomerName AS CustomerName1, B.CustomerName AS CustomerName2, A.City
FROM Customers A, Customers B
WHERE A.CustomerID <> B.CustomerID
AND A.City = B.City
ORDER BY A.City;
```

### UPDATE, изменение существующих строк

### DELETE, удаление строк

Операция, которая удаляет записи из таблицы, соответствующие заданному условию. При этом создаются логи удаления, то есть операцию можно отменить.

```
DELETE FROM table_name WHERE condition;
```

### FROM

### SET

### Вложенные запросы

SQL позволяет создавать вложенные запросы. Вложенный запрос – это запрос, размещенный внутри другого запроса SQL.

Вложенный запрос используется для выборки данных, которые будут использоваться в условии отбора записей основного запроса. Его применяют для:  
сравнения выражения с результатом вложенного запроса;  
определения того, включено ли выражение в результаты вложенного запроса;  
проверки того, выбирает ли запрос определенные строки.  

Вложенный запрос имеет следующие компоненты:  
ключевое слово SELECT, после которого указываются имена столбцов или выражения (чаще всего список содержит один элемент),  
ключевое слово FROM и имя таблицы, из которой выбираются данные,  
необязательное предложение WHERE,  
необязательное предложение GROUP BY,  
необязательное предложение HAVING.

опросы на собеседованиях очень часто бессистемные, они могут быть просто о том, что вопрошающему конкретно сейчас пришло в голову, и он вообще не факт, что сам умеет спрашивать о главном, а не второстепенном. Приведу только один пример у вас — про индексы. Вас, видимо, спрашивали о том, какие они бывают и каким синтаксисом создаются. Но поверьте, это совсем не самое главное из того, что стоит знать про индексы. Гораздо важнее понимать их суть, что они из себя представляют внутри — чтобы понимать, почему какой-то запрос индексом ускоряется, а какой-то нет.  

Изучение индексов для ускорения поиска данных в таблице  
Изучение ограничений для защиты данных и обеспечения целостности таблицы

Работа с представлениями и хранимыми процедурами  
Изучение представлений для создания виртуальных таблиц на основе запросов  
Изучение хранимых процедур для создания пользовательских функций и процессов

Продвинутые SQL скиллы:  
MVCC
Оптимизация запросов — знание структуры таблиц и индексов, а также понимания того, как оптимизировать запросы для улучшения производительности.  

Повышение производительности.
Основные понятия.
Методы просмотра таблиц.
Методы формирования соединений наборов строк.
Управление планировщиком.
Оптимизация запросов.

Работа с большими объемами данных — управление партиционированием, кластеризацией и другими методами для обработки и анализа больших объемов данных.  
Использование аналитических функций — использование функций, таких как RANK, ROW_NUMBER, LAG и LEAD, для выполнения сложных аналитических запросов.  
Работа с временными рядами данных — использование функций временных рядов, таких как DATE_TRUNC, DATE_PART и WINDOW функций, для анализа и управления временными рядами данных.  
Работа с географическими данными — использование специальных функций, таких как ST_Distance, ST_Within и ST_Intersection, для анализа и управления географическими данными.  
Работа с хранилищами данных — использование функций ETL (Extract, Transform, Load) для извлечения, преобразования и загрузки данных в хранилища данных.  
Работа с процедурами и триггерами — создание и управление процедурами и триггерами для автоматизации задач и обеспечения целостности данных.  
Работа с реляционной алгеброй — использование различных операторов, таких как JOIN, UNION, INTERSECT и EXCEPT, для выполнения сложных запросов.  
Работа с индексами — создание и управление индексами для улучшения производительности запросов.  
Работа с безопасностью данных — управление доступом к данным и защиту данных от несанкционированного доступа.  

Оконные функции


### Вложенные транзакции

Механизм, который неявно задействован при создании точек сохранения и при обработке исключений.

Что такое курсор и зачем он нужен?  
Что делает оператор JOIN, какие виды бывают?  
Что делает оператор HAVING, примеры?  
В каких случаях вы бы предпочли нереляционную БД?  
Что такое функциональный индекс?  

### VACUUM

Команда VACUUM высвобождает пространство, занимаемое «мертвыми» кортежами, что актуально для часто используемых таблиц. При обычных операциях в Postgres кортежи, удаленные или устаревшие в результаты обновления, физически не удаляются, а сохраняются в таблице до очистки.

### EXPLAIN, EXPLAIN ANALYZE

EXPLAIN ANALYZE – в отличие от просто EXPLAIN не только показывает план выполнения запроса, но и непосредственно выполняет запрос и показывает реальное время выполнения.

### Server side cursor

Способ работы с результатом запроса в базу данных, который позволяет не загружать весь объем данных в память, позволяет работать с большими объемами данных. Дополнительно углубленно можно поговорить про особенности работы в связке с pgbouncer.

Это любители mysql принесли 8 главу своей документации, где эта штука у них называется optimizer и состоит из нескольких частей (query planner, собственно query optimizer и т.д.)

В postgresql (и других базах) это называют planner и в русскоязычном IT в аспекте БД (не только postgres) принято слово именно что "планировщик". Ну а то, что некоторые любители mysql словом "планировщик" с давних пор называют почему-то event/job scheduler, вас в общем случае волновать не должно, это у них там своя атмосфера.

Это всё издержки переноса англоязычной терминологии на русский язык. Каким образом так случилось, что в русском scheduler стал планировщиком, я внятно объяснить не могу.

Schedule — это вообще-то график (дежурств), расписание (действий). Планирование как составление плана действий, здесь отсутствует, расписание — продукт планирования, а не процесс. И scheduler в этом смысле — это исполнитель расписания, а не его составитель.

Plan — это в некотором смысле алгоритм, последовательность шагов при исполнении какого-либо действия (из, например, расписания).

Optimize — в общем случае улучшение этого самого Plan.

Подытожим:

scheduler — хранит таблицу, расписание выполнения задач, и обеспечивает их запуск по этому расписанию;

planner — составляет план действий для исполнения конкретного запроса (а EXPLAIN его, этот план, показывает);

optimizer — часть planner'а (может и отсутствовать). Например, переупорядочивает действия, исключает повторные действия, и т.п.

Таким образом получается, что именно в PostgreSQL всё как раз таки и называется правильно.

### Источники

[Документация по PostgreSQL](https://www.postgresql.org/docs/).

Е. П. Моргунов. [PostgreSQL. Основы языка SQL](https://edu.postgrespro.ru/sql_primer.pdf).  
